<a href="https://colab.research.google.com/github/bull0330/hello-world/blob/master/2_SKT_OpenNMT_Pytorch_%EC%8B%A4%EC%8A%B5_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenNMT Pytorch를 이용한 기계번역 모델 만들어보기

***First Go to Runtime and  change the runtime type to GPU.***


<br>
 Copyright Park Chanjun
<br>
 Email: bcj1210@naver.com










# Git Clone

In [0]:
#!git clone https://github.com/OpenNMT/OpenNMT-py   # pytorch 버전

# Please install requirements.txt use by pip

> Error : You must restart the runtime in order to use newly installed versions.<br>
Solution : Click Restart Runtime => Redo



In [2]:
!pip3 install -r OpenNMT-py/requirements.txt --upgrade
!pip3 install configargparse --upgrade

  Cloning https://github.com/pytorch/text.git (to revision master) to /tmp/pip-req-build-niequjaq
  Running command git clone -q https://github.com/pytorch/text.git /tmp/pip-req-build-niequjaq
Requirement already up-to-date: six in /usr/local/lib/python3.6/dist-packages (from -r OpenNMT-py/requirements.txt (line 1)) (1.12.0)
Requirement already up-to-date: tqdm==4.30.* in /usr/local/lib/python3.6/dist-packages (from -r OpenNMT-py/requirements.txt (line 2)) (4.30.0)
Requirement already up-to-date: torch>=1.2 in /usr/local/lib/python3.6/dist-packages (from -r OpenNMT-py/requirements.txt (line 3)) (1.2.0)
Requirement already up-to-date: future in /usr/local/lib/python3.6/dist-packages (from -r OpenNMT-py/requirements.txt (line 5)) (0.17.1)
Requirement already up-to-date: configargparse in /usr/local/lib/python3.6/dist-packages (from -r OpenNMT-py/requirements.txt (line 6)) (0.14.0)
  Created wheel for torchtext: filename=torchtext-0.4.0-cp36-none-any.whl size=53452 sha256=d79614d9cdd96645

# 기계번역 모델 만드는 순서






**1.   데이터 수집**

병렬 코퍼스를 수집해야합니다.
이번 실습에서는 OpenNMT에서 제공하는 WMT 데이터를 사용합니다.

**2.   정제, 병렬 코퍼스 필터링**

병렬 코퍼스 필터링 (생략)


**3. 서브워드 분리**

BPE를 사용


**4. 학습**

GPU를 이용한 Transformer 훈련


**5. 번역**

모델을 이용한 번역


**6. Detokenization**

디토큰


**7. 점수계산**

BLEU 점수 계산

# Subword Tokenization

We use Byte Pair Encoding for Subword Tokenization

https://www.aclweb.org/anthology/P16-1162

i => input<br>
o ==> Output(*.code)<br>
s ==> Symbol<br>

learn_bpe ==> make code<br>
apply_bpe ==> apply subwordTokenization<br>

src-train, src-val,test ==> Need to apply src.code<br>
tgt-train,tgt-val ==> Need to apply tgt.code

In [0]:
!python3 OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src.code -s 10000

In [0]:
!python3 OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt.code -s 10000

In [0]:
!python3 OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src-train-bpe.txt

In [0]:
!python3 OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-val.txt -o OpenNMT-py/data/src-val-bpe.txt

In [0]:
!python3 OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-test.txt -o OpenNMT-py/data/src-test-bpe.txt

In [0]:
!python3 OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt-train-bpe.txt

In [0]:
!python3 OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-val.txt -o OpenNMT-py/data/tgt-val-bpe.txt

# **Preprocess the data**

We will be working with some example data in data/ folder.

The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space:

1. src-train.txt

2. tgt-train.txt

3. src-val.txt

4. tgt-val.txt


Train data and validataion data are required for machine translation training.

Validation files are required and used to evaluate the convergence of the training. It usually contains no more than 5000 sentences.


> If you think about it briefly, you can specify the path of train data and validation data, and specify the path and name to save in -save_data.

> If you want to set vocab size add below command
<br>
-src_vocab_size 32000 -tgt_vocab_size 32000

The vocab size is usually 32000.

In [10]:
!python3 OpenNMT-py/preprocess.py -train_src OpenNMT-py/data/src-train-bpe.txt -train_tgt OpenNMT-py/data/tgt-train-bpe.txt -valid_src OpenNMT-py/data/src-val-bpe.txt -valid_tgt OpenNMT-py/data/tgt-val-bpe.txt -save_data OpenNMT-py/data/demo -src_vocab_size 32000 -tgt_vocab_size 32000

Please backup existing pt files: OpenNMT-py/data/demo.train*.pt, to avoid overwriting them!


# **Train the data(Transformer)**

https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf


> If you get GPU-related errors, try halving batch_size

**Below is the full command, and if you want to know more about it, search about Transformer.**

!python OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model OpenNMT-py/data/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0  

In [11]:
!nvidia-smi

Thu Sep 12 02:55:57 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!python3 OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model OpenNMT-py/data/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0  # 오래걸리

[2019-09-12 02:56:00,743 INFO]  * src vocab size = 9739
[2019-09-12 02:56:00,743 INFO]  * tgt vocab size = 10034
[2019-09-12 02:56:00,743 INFO] Building model...
[2019-09-12 02:56:04,343 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(9739, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_featur

# **Translate**

Now that you have your model, you can start translating.

-model ==> Setting your model

Output predictions into pred.txt

In [0]:
!python3 OpenNMT-py/translate.py -model OpenNMT-py/data/model/model_step_1000.pt -src OpenNMT-py/data/src-val.txt -output OpenNMT-py/data/pred.txt -replace_unk -verbose -gpu 0  # pred.txt 가 여기서 만들어짐

# Detokenization

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.

We Use "sed" for BPE Detokenization


In [0]:
!sed -i "s/@@ //g"  OpenNMT-py/data/pred.txt  #일종의 정규표현식. "@@ " 부분을 공백으로 처리해서 detokenization

# Evaluation Using BLEU

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

https://www.aclweb.org/anthology/P02-1040

In [0]:
!perl  OpenNMT-py/tools/multi-bleu.perl OpenNMT-py/data/tgt-val.txt < OpenNMT-py/data/pred.txt

If you have Any Question Please Email to  "bcj1210@naver.com"